In [284]:
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [285]:
# loading the dataset
df = pd.read_csv('/content/breast-cancer.csv')

In [286]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [287]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [288]:
df.isnull().sum()

,0
id,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0


In [289]:
df.duplicated().sum()

0

In [290]:
df.shape

(569, 32)

In [291]:
df.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [292]:
# checking the frequency of the label
df['diagnosis'].value_counts()

,count
diagnosis,
B,357
M,212


In [293]:
# droping the irrelevent column
df.drop(columns=['id'],inplace=True)

In [294]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [295]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [296]:
# splitting the dataset
X_train,X_test , y_train,y_test = train_test_split(X,y,test_size=0.2)

In [297]:
X_train.shape , X_test.shape,y_train.shape,y_test.shape

((455, 30), (114, 30), (455,), (114,))

In [298]:
# normalize the input features using the standard scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [299]:
X_train[:5] , y_train[:5]

(array([[-0.15510514, -0.67723933, -0.1982989 , -0.25229634, -0.26867511,
         -0.59833985, -0.88241716, -0.76830231, -0.66038675, -0.16727463,
         -0.70379554, -1.33848349, -0.71442796, -0.49619859, -1.07595845,
         -0.98816923, -0.7853251 , -1.07059509, -1.19287206, -0.75444719,
         -0.11595413, -0.89222175, -0.19317281, -0.21569418, -0.29286415,
         -0.61399589, -0.72503778, -0.42333078, -0.13936027, -0.3357641 ],
        [ 0.18463653, -1.00717803,  0.14640519,  0.05104072, -1.1239766 ,
         -0.35374485, -0.42182244, -0.43417435, -0.45029517, -0.88119106,
         -0.56565953, -1.42433189, -0.49997487, -0.37162574, -1.25933624,
         -0.40666176, -0.27367614, -0.53100909, -1.07782407, -0.50555099,
          0.15030896, -1.21766371,  0.10556528,  0.00878917, -1.17003445,
          0.152775  ,  0.18778152, -0.01186224, -0.33251546, -0.06315604],
        [ 0.04764392,  0.1334672 ,  0.06826169, -0.06985843,  0.075912  ,
          0.18704827,  0.04631305,  

In [300]:
# transforming the labels through LabelEncoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [301]:
y_train[:5]

array([0, 0, 1, 0, 0])

In [302]:
# Numpy arrays to pytorch tensors

X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [303]:
X_train.shape

(455, 30)

In [304]:
len(X_train)

455

In [305]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]

In [306]:
train_dataset = CustomDataset(X_train_tensor,y_train_tensor)
test_dataset = CustomDataset(X_test_tensor,y_test_tensor)

In [307]:
train_loader = DataLoader(train_dataset ,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset ,batch_size=32,shuffle=True)

In [308]:
for inputs, labels  in train_loader:
    print(inputs.shape, labels.shape)
    print(inputs.dtype , labels.dtype)
    break  # Only print the first batch


torch.Size([32, 30]) torch.Size([32])
torch.float32 torch.float32


In [309]:
# Defining the model
import torch.nn as nn

class SimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    output = self.linear(features)
    output = self.sigmoid(output)

    return output

In [310]:
# creating the model of the class
model_1 = SimpleNN(X_train_tensor.shape[1])
model_1

SimpleNN(
  (linear): Linear(in_features=30, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [311]:
# define optimizer
optimizer = torch.optim.SGD(model_1.parameters(), lr=0.1)

# define loss function
loss_fn = nn.BCELoss()

In [312]:
epochs = 25
# Training loop
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    model_1.train()
    # forward pass
    y_pred = model_1(batch_features)

    # loss calculate
    loss = loss_fn(y_pred, batch_labels.view(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1} | Loss: {loss.item()}')

Epoch: 1 | Loss: 0.19084720313549042
Epoch: 2 | Loss: 0.17644596099853516
Epoch: 3 | Loss: 0.10431314259767532
Epoch: 4 | Loss: 0.10234277695417404
Epoch: 5 | Loss: 0.1277780681848526
Epoch: 6 | Loss: 0.07923432439565659
Epoch: 7 | Loss: 0.04426288232207298
Epoch: 8 | Loss: 0.22221648693084717
Epoch: 9 | Loss: 0.04543157294392586
Epoch: 10 | Loss: 0.01155961211770773
Epoch: 11 | Loss: 0.002529528457671404
Epoch: 12 | Loss: 0.10787045210599899
Epoch: 13 | Loss: 0.14768488705158234
Epoch: 14 | Loss: 0.015841713175177574
Epoch: 15 | Loss: 0.07312732189893723
Epoch: 16 | Loss: 0.250397264957428
Epoch: 17 | Loss: 0.01254955492913723
Epoch: 18 | Loss: 0.008285393007099628
Epoch: 19 | Loss: 0.07188506424427032
Epoch: 20 | Loss: 0.012075130827724934
Epoch: 21 | Loss: 0.1415693610906601
Epoch: 22 | Loss: 0.09396976232528687
Epoch: 23 | Loss: 0.005285545252263546
Epoch: 24 | Loss: 0.13828544318675995
Epoch: 25 | Loss: 0.012596981599926949


In [313]:
# Model evaluation using test_loader
model_1.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.inference_mode():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model_1(batch_features)
        y_pred = (y_pred > 0.8).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')

Accuracy: 0.9844
